In [1]:
import cv2
import os

# Define labels (folders to save images)
labels = ['1', '2', '3', '4', '5', 'a', 'e', 'h', 'i', 'l', 'm', 'o', 's', 'r']
dataset_path = "ISL"

# Create dataset folders if they don't exist
for label in labels:
    os.makedirs(os.path.join(dataset_path, label), exist_ok=True)

# Open webcam
cap = cv2.VideoCapture(0)

current_label = 0  # Start with the first label
image_count = 0  # Track number of images per label

print(f"Show hand sign for label: {labels[current_label]} (Press SPACE to capture, 'n' for next)")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Show instructions on screen
    cv2.putText(frame, f"Label: {labels[current_label]}", (10, 50), 
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow("Hand Sign Capture", frame)

    key = cv2.waitKey(1) & 0xFF

    if key == ord(' '):  # Capture image when SPACE is pressed
        image_path = os.path.join(dataset_path, labels[current_label], f"{image_count}.jpg")
        cv2.imwrite(image_path, frame)
        print(f"Saved: {image_path}")
        image_count += 1

        if image_count >= 20:  # Move to next label after 20 images
            image_count = 0
            current_label += 1
            if current_label >= len(labels):
                print("All hand signs captured!")
                break
            print(f"Next: Show hand sign for label: {labels[current_label]}")

    elif key == ord('n'):  # Skip to next label manually
        image_count = 0
        current_label += 1
        if current_label >= len(labels):
            print("All hand signs captured!")
            break
        print(f"Next: Show hand sign for label: {labels[current_label]}")

    elif key == ord('q'):  # Quit if 'q' is pressed
        break

cap.release()
cv2.destroyAllWindows()

Show hand sign for label: 1 (Press SPACE to capture, 'n' for next)
Next: Show hand sign for label: 2
Next: Show hand sign for label: 3
Next: Show hand sign for label: 4
Next: Show hand sign for label: 5
Next: Show hand sign for label: a
Next: Show hand sign for label: e
Next: Show hand sign for label: h
Next: Show hand sign for label: i
Next: Show hand sign for label: l
Next: Show hand sign for label: m
Next: Show hand sign for label: o
Next: Show hand sign for label: s
Next: Show hand sign for label: r
All hand signs captured!


In [1]:
import os
import pickle
import mediapipe as mp
import cv2
import pickle
import numpy as np
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.3)

DATA_DIR = 'dataset'
data = []
labels = []
label_map = {}
label_index = 0
valid_extensions = ['.jpg', '.jpeg', '.png']

for dir_ in os.listdir(DATA_DIR):
    for img_path in os.listdir(os.path.join(DATA_DIR, dir_)):
        if not any(img_path.lower().endswith(ext) for ext in valid_extensions):
            continue

        img = cv2.imread(os.path.join(DATA_DIR, dir_, img_path))
        if img is None:
            continue

        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = hands.process(img_rgb)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                data_aux = []
                x_ = []
                y_ = []

                for lm in hand_landmarks.landmark:
                    x_.append(lm.x)
                    y_.append(lm.y)

                for lm in hand_landmarks.landmark:
                    data_aux.append(lm.x - min(x_))
                    data_aux.append(lm.y - min(y_))

                data.append(data_aux)
                if dir_ not in label_map:
                    label_map[dir_] = label_index
                    label_index += 1
                labels.append(label_map[dir_])

# Save data and label map
with open('data.pickle', 'wb') as f:
    pickle.dump({'data': data, 'labels': labels, 'label_map': label_map}, f)

print('Data saved to data.pickle')
print('Label map saved to label_map.pickle')
print('Label map:', label_map)

Data saved to data.pickle
Label map saved to label_map.pickle
Label map: {'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, 'a': 5, 'e': 6, 'h': 7, 'i': 8, 'l': 9, 'm': 10, 'o': 11, 'r': 12, 's': 13}


In [2]:
import pickle
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load preprocessed data
with open('data.pickle', 'rb') as f:
    data_dict = pickle.load(f)

data = np.asarray(data_dict['data'])
labels = np.asarray(data_dict['labels'])

# Optional: keep the label_map for decoding predictions later
label_map = data_dict.get('label_map', None)

# Split data
x_train, x_test, y_train, y_test = train_test_split(
    data, labels, test_size=0.2, stratify=labels, random_state=42, shuffle=True
)

# Train model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(x_train, y_train)

# Evaluate
y_predict = model.predict(x_test)
score = accuracy_score(y_test, y_predict)

print(f'{score * 100:.2f}% of samples were classified correctly!')

# Save model and label map
with open('model.p', 'wb') as f:
    pickle.dump({'model': model, 'label_map': label_map}, f)

print('Model saved to model.p')

100.00% of samples were classified correctly!
Model saved to model.p


In [3]:
import pickle
import cv2
import mediapipe as mp
import numpy as np

# ------------------- Load Model -----------------------
with open('./model.p', 'rb') as f:
    model_dict = pickle.load(f)

model = model_dict['model']
label_map = model_dict.get('label_map')

# Invert label_map for predictions (int -> label)
if label_map:
    labels_dict = {v: k for k, v in label_map.items()}
else:
    labels_dict = {'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, 'a': 5, 'e': 6, 'h': 7, 'i': 8, 'l': 9, 'm': 10, 'o': 11, 'r': 12, 's': 13}  # Fallback

# ------------------- Feature Extraction -----------------------
def extract_landmark_features(landmarks):
    x_vals = [lm.x for lm in landmarks]
    y_vals = [lm.y for lm in landmarks]
    min_x, min_y = min(x_vals), min(y_vals)

    features = []
    for lm in landmarks:
        features.append(lm.x - min_x)
        features.append(lm.y - min_y)

    return features, x_vals, y_vals

# ------------------- Initialize Camera -----------------------
cap = cv2.VideoCapture(0)

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
hands = mp_hands.Hands(static_image_mode=False, min_detection_confidence=0.5)

# ------------------- Main Loop -----------------------
while True:
    ret, frame = cap.read()
    if not ret:
        continue

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    H, W, _ = frame.shape

    results = hands.process(frame_rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Draw hand landmarks
            mp_drawing.draw_landmarks(
                frame, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style()
            )

            features, x_vals, y_vals = extract_landmark_features(hand_landmarks.landmark)

            # Predict only if feature length matches model input
            if len(features) == model.n_features_in_:
                prediction = model.predict([np.asarray(features)])
                predicted_char = labels_dict[int(prediction[0])]

                # Bounding box
                x1, y1 = int(min(x_vals) * W) - 20, int(min(y_vals) * H) - 20
                x2, y2 = int(max(x_vals) * W) + 20, int(max(y_vals) * H) + 20

                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 0), 2)
                cv2.putText(frame, predicted_char, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 255, 0), 2, cv2.LINE_AA)
            else:
                print(f"⚠️ Feature mismatch: expected {model.n_features_in_}, got {len(features)}")

    # Show the frame
    cv2.imshow('Hand Sign Detection', frame)

    # Exit on 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()